In [ ]:
# Imports and path setup
import sys
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

ROOT = Path.cwd()
DATA_PROCESSED = ROOT / 'data' / 'processed'
MODELS = ROOT / 'models'
OUTPUTS = ROOT / 'outputs'
RESULTS = OUTPUTS / 'results'
GRAPHS = OUTPUTS / 'graphs'
for p in [DATA_PROCESSED, MODELS, RESULTS, GRAPHS]:
    p.mkdir(parents=True, exist_ok=True)

X_TEST_IN = DATA_PROCESSED / 'X_test.csv'
Y_TEST_IN = DATA_PROCESSED / 'y_test.csv'
MODEL_IN = MODELS / 'car_price_model.pkl'
METRICS_OUT = RESULTS / 'evaluation_metrics.csv'
REPORT_OUT = RESULTS / 'model_evaluation_report.txt'
PREDICTIONS_OUT = RESULTS / 'evaluation_predictions.csv'
PLOT_PRED_ACT = GRAPHS / 'eval_predicted_vs_actual.png'
PLOT_RESID = GRAPHS / 'eval_residuals_hist.png'
PLOT_CAL = GRAPHS / 'eval_calibration.png'

print('Evaluation inputs:', X_TEST_IN, Y_TEST_IN, MODEL_IN)

In [ ]:
# Load model and test data with error handling
try:
    with open(MODEL_IN, 'rb') as f:
        model = pickle.load(f)
except FileNotFoundError:
    sys.exit(f'ERROR: Model file not found at {MODEL_IN}. Run training first.')
except Exception as e:
    sys.exit(f'ERROR loading model: {e}')

try:
    X_test = pd.read_csv(X_TEST_IN)
    y_test = pd.read_csv(Y_TEST_IN).squeeze()
except FileNotFoundError as e:
    sys.exit(f'ERROR: Test dataset missing: {e}')
except Exception as e:
    sys.exit(f'ERROR reading test data: {e}')

print('Loaded model and test data shapes:', getattr(X_test, 'shape', None), getattr(y_test, 'shape', None))

In [ ]:
# Predictions and metrics
try:
    preds = model.predict(X_test)
except Exception as e:
    sys.exit(f'ERROR during prediction: {e}')

# Ensure arrays
y_true = np.asarray(y_test).ravel()
y_pred = np.asarray(preds).ravel()

r2 = r2_score(y_true, y_pred)
mae = mean_absolute_error(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
# Avoid division by zero in MAPE
mape = np.mean(np.abs((y_true - y_pred) / np.where(y_true==0, 1e-8, y_true))) * 100

metrics_df = pd.DataFrame([{
    'mae': mae,
    'rmse': rmse,
metrics_df.to_csv(METRICS_OUT, index=False)
print('Saved evaluation metrics to', METRICS_OUT)
display(metrics_df)

# Save predictions with true values
pred_df = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred})
pred_df.to_csv(PREDICTIONS_OUT, index=False)
print('Saved predictions to', PREDICTIONS_OUT)

In [ ]:
# Plots: Predicted vs Actual, Residuals, Calibration (binned)
try:
    plt.figure(figsize=(7,5))
    plt.scatter(y_true, y_pred, alpha=0.6)
    mn = min(y_true.min(), y_pred.min())
    mx = max(y_true.max(), y_pred.max())
    plt.plot([mn, mx], [mn, mx], 'r--')
    plt.xlabel('Actual Selling_Price')
    plt.ylabel('Predicted Selling_Price')
    plt.title('Predicted vs Actual (Test)')
    plt.tight_layout()
    plt.savefig(PLOT_PRED_ACT, dpi=150)
    plt.show()
    print('Saved', PLOT_PRED_ACT)

    residuals = y_true - y_pred
    plt.figure(figsize=(7,4))
    plt.hist(residuals, bins=40, edgecolor='k')
    plt.title('Residuals Distribution (Test)')
    plt.xlabel('Residual')
    plt.tight_layout()
    plt.savefig(PLOT_RESID, dpi=150)
    plt.show()
    print('Saved', PLOT_RESID)

    # Simple calibration plot: bin by predicted value and plot mean(true) vs mean(pred)
    bins = pd.qcut(y_pred, q=10, duplicates='drop')
    calib = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred, 'bin': bins})
    calib_group = calib.groupby('bin').agg({'y_true': 'mean', 'y_pred': 'mean'}).reset_index()
    plt.figure(figsize=(6,5))
    plt.plot(calib_group['y_pred'], calib_group['y_true'], 'o-')
    plt.plot([calib_group['y_pred'].min(), calib_group['y_pred'].max()], [calib_group['y_pred'].min(), calib_group['y_pred'].max()], 'r--')
    plt.xlabel('Mean Predicted')
    plt.ylabel('Mean Actual')
    plt.title('Calibration by Predicted Quantile')
    plt.tight_layout()
    plt.savefig(PLOT_CAL, dpi=150)
    plt.show()
    print('Saved', PLOT_CAL)
except Exception as e:
    print('Could not create/save evaluation plots:', e)

In [ ]:
# Try to save feature importances if model exposes them and feature names are available
try:
    if hasattr(model, 'feature_importances_') and hasattr(X_test, 'columns'):
        fi = model.feature_importances_
        fi_df = pd.DataFrame({'feature': X_test.columns.tolist(), 'importance': fi}).sort_values('importance', ascending=False)
        fi_df.to_csv(RESULTS / 'evaluation_feature_importances.csv', index=False)
        print('Saved feature importances to', RESULTS / 'evaluation_feature_importances.csv')
except Exception as e:
    print('Could not save feature importances:', e)

In [ ]:
# Write a simple human-readable report
try:
    with open(REPORT_OUT, 'w', encoding='utf-8') as f:
        f.write('Model Evaluation Report
')
        f.write('====================
')
        f.write(f'R2: {r2:.4f}
')
        f.write(f'MAE: {mae:.4f}
')
        f.write(f'MSE: {mse:.4f}
')
        f.write(f'RMSE: {rmse:.4f}
')
        f.write(f'MAPE (%): {mape:.4f}
')
        f.write('
Notes: All paths are relative. For more details see evaluation_metrics.csv and evaluation_predictions.csv')
    print('Saved evaluation report to', REPORT_OUT)
except Exception as e:
    print('Could not write report file:', e)

## End of Evaluation

Saved outputs:
- `outputs/results/evaluation_metrics.csv`
- `outputs/results/evaluation_predictions.csv`
- `outputs/results/model_evaluation_report.txt`
- `outputs/graphs/` plots: `eval_predicted_vs_actual.png`, `eval_residuals_hist.png`, `eval_calibration.png`
- (optional) `outputs/results/evaluation_feature_importances.csv` if available from model.